In [1]:
import numpy as np
import pandas as pd
from time import perf_counter
from bisect import insort_left
from collections import deque
from itertools import combinations, product
from sortedcontainers import SortedSet


import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [2]:
def random_search(distance_matrix):
    n = len(distance_matrix)
    solution = list(range(n))
    np.random.shuffle(solution)
    
    return solution[:(n//2)]

In [3]:
def calculate_euclidean_distance(node1, node2):
    return np.round(np.sqrt((node1['x'] - node2['x'])**2 + (node1['y'] - node2['y'])**2))


def get_distance_matrix(df, costs=False):
    num_nodes = df.shape[0]
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        node1 = df.iloc[i]
        for j in range(i, num_nodes):
            node2 = df.iloc[j]
            distance = calculate_euclidean_distance(node1, node2)
            if costs:
                distance_matrix[i,j] = distance + node2['cost']
                distance_matrix[j,i] = distance + node1['cost']
            else:
                distance_matrix[i,j] = distance
                distance_matrix[j,i] = distance
    return distance_matrix

In [4]:
def compute_inter_move_delta(solution, distance_matrix, costs, idx, new_node):
    n = len(solution)
    new_solution = solution.copy()
    old_node = solution[idx]
#     print(new_node, old_node)
    new = (costs[new_node] +
            distance_matrix[new_solution[idx-1], new_node] +
            distance_matrix[new_node, new_solution[(idx+1)%n]])

    old = (costs[old_node] +
             distance_matrix[new_solution[idx-1], old_node] +
             distance_matrix[old_node, new_solution[(idx+1)%n]])

    delta = new - old
    new_solution[idx] = new_node

    return new_solution, delta

In [5]:
def compute_intra_move_delta(solution, distance_matrix, indices, backward=False):
    n = len(solution)
    new_solution = solution.copy()
    i, j = indices[0], indices[1]
    if i > j: 
        i, j = j, i

    if (i + 1) % n == j % n or (j + 1) % n == i % n:
        return new_solution, 0
    
    new, old = 0, 0
    if backward:
        new = distance_matrix[new_solution[i], new_solution[j]] + distance_matrix[new_solution[i-1], new_solution[j-1]]
        old = distance_matrix[new_solution[i-1], new_solution[i]] + distance_matrix[new_solution[j-1], new_solution[j]]
    else:
        new = distance_matrix[new_solution[i], new_solution[j]] + distance_matrix[new_solution[i+1], new_solution[int(j % n)]]
        old = distance_matrix[new_solution[i], new_solution[i+1]] + distance_matrix[new_solution[j], new_solution[int(j % n)]]

    delta = new - old
    if backward:
        if i == 0:
            new_solution = np.concatenate((np.flip(new_solution[i:j]), new_solution[j:]))
        else:
            new_solution = np.concatenate((new_solution[:i], np.flip(new_solution[i:j]), new_solution[j:]))
    else:
        if j == n - 1:
            new_solution = np.concatenate((new_solution[:i], np.flip(new_solution[i+1:j+1])))
        else:
            new_solution = np.concatenate((new_solution[:i], np.flip(new_solution[i+1:j+1]), new_solution[j+1:]))

    return new_solution.astype(int), delta

In [6]:
def make_move(solution, move, distance_matrix, costs):
    solution = list(solution)
    if move[0] == 'node_swap':
        idx, new_node = move[1]
        idx = solution.index(idx)
        new_solution, _ = compute_inter_move_delta(solution, distance_matrix, costs, idx, new_node)
    elif move[0] == 'edge_swap':
        _, node_pair_indeces, backward, _ = move
        node_pair_indeces = (solution.index(node_pair_indeces[0]), solution.index(node_pair_indeces[1]))
        new_solution, _ = compute_intra_move_delta(solution, distance_matrix, node_pair_indeces, backward)
    return new_solution

In [7]:
def edge_exists_order_preserved(nodes, solution):
    if nodes[0] not in solution or nodes[1] not in solution:
        return False, False
    
    pos_1 = np.where(solution == nodes[0])[0][0]
    pos_2 = np.where(solution == nodes[1])[0][0]

    edge_exists = abs(pos_1 - pos_2) == 1
    order_preserved = pos_1 + 1 == pos_2
    return edge_exists, order_preserved



def check_move(move, solution):
    node1, node2 = move[1]

    if move[0] == "node_swap":
        return (node1 not in solution) and (node2 in solution), False
    
    node_intra1, node_intra2 =  move[3]
    if move[0] == "edge_swap" and not move[2]:
        edge1 = [node1, node_intra1]
        edge2 = [node2, node_intra2]
    elif move[0] == "edge_swap" and move[2]:
        edge1 = [node_intra1, node1]
        edge2 = [node_intra2, node2]

    edge_exists1, order_preserved1 = edge_exists_order_preserved(np.array(edge1), solution)
    edge_exists2, order_preserved2 = edge_exists_order_preserved(np.array(edge2), solution)

    if edge_exists1 and edge_exists2:
        return (order_preserved1 and order_preserved2), (not order_preserved1 or not order_preserved2)
    
    return False, False

In [8]:
def steepest_local_search_deltas(solution, distance_matrix, costs):
    LM = SortedSet()    # [ (delta, ('move_type', (node1, node2), bool | None)) ]
    n = len(solution)
    
    while True:
        total_cost = sum(distance_matrix_with_costs[solution[iii]][solution[iii+1]] for iii in range(len(solution) - 1))
        total_cost += distance_matrix_with_costs[solution[-1]][solution[0]]
        print(total_cost, len(LM))

        # intra edge moves
        for node_pair_indeces in combinations(range(100), 2):
            
            _, delta = compute_intra_move_delta(solution, distance_matrix, node_pair_indeces, False)
            if delta < 0:
                node_pair = (solution[node_pair_indeces[0]], solution[node_pair_indeces[1]])
                adjacent_node_pair = (solution[node_pair_indeces[0] % n], solution[node_pair_indeces[1] % n])
                LM.add( (delta, ('edge_swap', node_pair, False, adjacent_node_pair)) )
                
            _, delta = compute_intra_move_delta(solution, distance_matrix, node_pair_indeces, True)
            if delta < 0:
                node_pair = (solution[node_pair_indeces[0]], solution[node_pair_indeces[1]])
                adjacent_node_pair = (solution[(node_pair_indeces[0] - 2) % n], solution[(node_pair_indeces[1] - 2) % n])
                LM.add( (delta, ('edge_swap', node_pair, True, adjacent_node_pair)) )
        
        # inter node moves
        outer_nodes_set = set(range(len(distance_matrix))) - set(solution)     
        for in_out_node_pair in product(range(100), outer_nodes_set):
            idx, new_node = in_out_node_pair
            # print(len(solution))
            _, delta = compute_inter_move_delta(solution, distance_matrix, costs, idx, new_node)
            if delta < 0:
                node_pair = (solution[idx], new_node)
                LM.add( (delta, ('node_swap', node_pair)) )


        idx = 0
        while idx < len(LM):
            move = LM[idx]
            is_applicable, store = check_move(move[1], solution)
            
            flag = False
            if is_applicable:
                flag = True
                solution = make_move(solution, move[1], distance_matrix, costs)
                LM.remove(move)
                # break 
            
            if not store and not flag:
                LM.remove(move)
                
            if flag:
                flag = False
                break
                
            idx += 1
        
        
        if len(LM) == 0:
            break
        
    return solution

In [9]:
df = pd.read_csv('../data/TSPA.csv', names=['x', 'y', 'cost'], sep=';')
costs = df.cost
distance_matrix = get_distance_matrix(df)
distance_matrix_with_costs = get_distance_matrix(df, True)
solutionn = random_search(distance_matrix)

In [10]:
total_cost = sum(distance_matrix_with_costs[solutionn[i]][solutionn[i+1]] for i in range(len(solutionn) - 1))
total_cost += distance_matrix_with_costs[solutionn[-1]][solutionn[0]]
total_cost

257273.0

In [11]:
solutionnn = steepest_local_search_deltas(solutionn, distance_matrix, costs)

257273.0 0
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391
257273.0 3391


KeyboardInterrupt: 

In [ ]:
total_cost = sum(distance_matrix_with_costs[solutionnn[i]][solutionnn[i+1]] for i in range(len(solutionnn) - 1))
total_cost += distance_matrix_with_costs[solutionnn[-1]][solutionnn[0]]
total_cost